# Climate change sucks

In [11]:
import csv
import numpy as np

file = open('./combined_dataset.csv')
# file = open('./Annual_Surface_Temperature_Change.csv')
csvreader = csv.reader(file)

countries = []  # To store country names
numerical_data = []  # To store numerical data

next(csvreader, None)

for row in csvreader:
    # Split the data into country name and numerical part
    country_name = row[1]  # First column
    numerical_part = []
    for value in row[6:-2]:
        try:
            numerical_part.append(float(value))
        except ValueError:
            numerical_part.append(None)  

    countries.append(country_name)
    numerical_data.append(numerical_part)

# Convert the data lists to numpy arrays
countries = np.array(countries)
numerical_data = np.array(numerical_data, dtype=float)

print(countries.shape)
print(numerical_data.shape)

(2226,)
(2226, 29)


In [16]:
import csv
import numpy as np

file = open('./combined_dataset.csv')
csvreader = csv.reader(file)

# Skip the header row
next(csvreader, None)

# Initialize a nested dictionary to store data for each country, indicator, and data
country_data = {}

for row in csvreader:
    country_name = row[1]
    indicator = row[4]  # Indicator column
    year_data = row[6:]  # Data from 1992 onwards
    year_data = [float(value) if value != '' else None for value in year_data]

    # Check if the country is already in the dictionary
    if country_name in country_data:
        if indicator in country_data[country_name]:
            country_data[country_name][indicator].append(year_data)
        else:
            country_data[country_name][indicator] = [year_data]
    else:
        country_data[country_name] = {
            indicator: [year_data]
        }

# Convert the lists of data for each country to NumPy arrays
for country, indicators_data in country_data.items():
    for indicator, data in indicators_data.items():
        indicators_data[indicator] = np.array(data, dtype=float)
        
# Example: Accessing data for a specific country, indicator, and year
desired_country = "Afghanistan, Islamic Rep. of"
desired_indicator = "Temperature change"
desired_year_index = 0  # Index for the year you want, e.g., 0 for F1992

if desired_country in country_data:
    country_info = country_data[desired_country]
    if desired_indicator in country_info:
        data_for_indicator = country_info[desired_indicator]
        data_for_year = data_for_indicator[:, desired_year_index]
        print(data_for_year)
    else:
        print(f"Data not found for the indicator: {desired_indicator}")
else:
    print(f"Data not found for {desired_country}")


[-0.294]


In [40]:
# Evaluate the trained models on the validation set
for country in val_countries:
    if any(indicator.startswith("Temperature change") for indicator in country_data[country].keys()):
        # Find the indicator that starts with "Temperature change"
        temperature_change_indicator = next(
            (indicator for indicator in country_data[country].keys() if indicator.startswith("Temperature change")),
            None
        )

        if temperature_change_indicator is not None:
            model = trained_models[country]
            input_data = torch.Tensor(country_data[country][temperature_change_indicator][:, :-2])
            target_data = torch.Tensor(country_data[country][temperature_change_indicator][:, -1])
            with torch.no_grad():
                output = model(input_data)
                loss = criterion(output, target_data)
                evaluation_results[country] = loss.item()

# Example: Print the evaluation results
for country, loss in evaluation_results.items():
    print(f"Validation loss for {country}: {loss}")


KeyError: 'Bhutan'

In [16]:
! pip install torch

                                              0.0/192.3 MB ? eta -:--:--
                                              0.0/192.3 MB ? eta -:--:--
                                              0.0/192.3 MB ? eta -:--:--
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                             0.0/192.3 MB 81.9 kB/s eta 0:39:08
                                             0.0/192.3 MB 81.9 kB/s eta 0:39:08
                                            0.1/192.3 MB 117.2 kB/s eta 0:27:22
                                            0.1/192.3 MB 148.

In [119]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

import pennylane as qml

# Define a quantum device
dev = qml.device("default.qubit", wires=1)

# Define a quantum circuit using Pennylane
@qml.qnode(dev)
def quantum_circuit(x):
    qml.RY(x[0], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define a Pennylane hybrid model that combines the quantum circuit and PyTorch model
class HybridModel(nn.Module):
    def __init__(self, input_size):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128).to(torch.float64)
        self.fc2 = nn.Linear(128, 64).to(torch.float64)
        self.fc3 = nn.Linear(64, 1).to(torch.float64)
        self.quantum_layer = quantum_circuit

    def forward(self, x):
        x1 = torch.relu(self.fc1(x))
        x2 = torch.relu(self.fc2(x1))
        output = self.fc3(x2)

        # Pass input through the quantum layer
        quantum_output = self.quantum_layer(x)
        output = output + quantum_output

        return output

# Instantiate the regression model
input_size = 30
model = HybridModel(input_size)

# Define a loss function (Mean Squared Error) and an optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

file = open('./Annual_Surface_Temperature_Change.csv')
csvreader = csv.reader(file)

prev_years = []
final_years = []
first = True
for row in csvreader:
    if row[0] == '225':
        prev_years.append([float(val) if val != "" else 0.0 for val in row[35:len(row)-1]])
        final_years.append([float(row[len(row)-1]) if row[len(row)-1] != "" else 0.0])
        break

    
    prev_years.append([float(val) if val != "" else 0.0 for val in row[35:len(row)-1]])
    final_years.append([float(row[len(row)-1]) if row[len(row)-1] != "" else 0.0])

print(len(prev_years))
file.close()

prev_years = torch.tensor(prev_years, dtype=torch.float64)
final_years = torch.tensor(final_years, dtype=torch.float64)
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(prev_years)
    loss = criterion(output, final_years)
    loss.backward()
    optimizer.step()

# After training, you can use the model to make predictions
model.eval()
for i in range(10):
    with torch.no_grad():
        test_input = prev_years[i]
        predicted_output = model(test_input)

    print(f"The predicted value is: {predicted_output.item()}")
    print(f"The actual value is: {final_years[i][0]}")
    print(abs(predicted_output.item()-final_years[i][0]))

200


c:\Users\User\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([200, 1])) that is different to the input size (torch.Size([200, 30])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([30])
The predicted value is: 2.4142025131135454
The actual value is: 2.012
tensor(0.4022, dtype=torch.float64)
torch.Size([30])
The predicted value is: 2.173856760779297
The actual value is: 1.518
tensor(0.6559, dtype=torch.float64)
torch.Size([30])
The predicted value is: 2.1702898770573933
The actual value is: 1.688
tensor(0.4823, dtype=torch.float64)
torch.Size([30])
The predicted value is: 1.616659742558023
The actual value is: 1.256
tensor(0.3607, dtype=torch.float64)
torch.Size([30])
The predicted value is: 1.4931697057285644
The actual value is: 1.212
tensor(0.2812, dtype=torch.float64)
torch.Size([30])
The predicted value is: 1.2051208455771585
The actual value is: 0.839
tensor(0.3661, dtype=torch.float64)
torch.Size([30])
The predicted value is: 1.1048365219356397
The actual value is: 0.77
tensor(0.3348, dtype=torch.float64)
torch.Size([30])
The predicted value is: 0.8974759618537234
The actual value is: 0.643
tensor(0.2545, dtype=torch.float64)
torch.Size([30])
Th

In [109]:
import torch 
# Initialize empty lists to store data
prev_years = [[],[]]
final_years = []

# Load temperature change data
with open('./Annual_Surface_Temperature_Change.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Assuming the data starts from column 4 and ends at the last column (index -1)
        temperature_data = [float(val) if val else 0.0 for val in row[35:-1]]
        final_temperature = float(row[-1]) if row[-1] else 0.0
        prev_years[0].append(temperature_data)
        final_years.append(final_temperature)

# Load total number of disasters data
with open("./disastors.csv", 'r', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter='\t')  # Set delimiter to '\t' for tab-separated values

    for row in csvreader:
        if row[4] == 'Total disasters' and row[5] == 'Number of':
            # Assuming the data starts from column 6 and ends at the last column (index -1)
            disasters_data = [float(val) if val else 0.0 for val in row[6:-1]]
            final_disasters = float(row[-1]) if row[-1] else 0.0
            prev_years[1].append(disasters_data)
            final_years.append(final_disasters)

# Convert the lists to PyTorch tensors
prev_years = torch.tensor(prev_years, dtype=torch.float64)
final_years = torch.tensor(final_years, dtype=torch.float64)

print(prev_years.shape)
print(final_years.shape)


torch.Size([2, 200, 30])
torch.Size([400])


In [126]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tqdm
import pennylane as qml
import tensorflow as tf

# Define a quantum device
dev = qml.device("default.qubit", wires=1)

# Define a quantum circuit using Pennylane
@qml.qnode(dev)
def quantum_circuit(x):
    qml.RY(x[0], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define a Pennylane hybrid model that combines the quantum circuit and PyTorch model
class HybridModel(nn.Module):
    def __init__(self, input_size):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128).to(torch.float64)
        self.fc2 = nn.Linear(128, 64).to(torch.float64)
        self.fc3 = nn.Linear(64, 1).to(torch.float64)
        self.quantum_layer = quantum_circuit

    def forward(self, x):
        x1 = torch.relu(self.fc1(x))
        x2 = torch.relu(self.fc2(x1))
        output = self.fc3(x2)

        # Pass input through the quantum layer
        quantum_output = self.quantum_layer(x)
        output = output + quantum_output

        return output

# Load and preprocess your temperature change and total number of disasters data
# Assuming you have organized the data and loaded it into prev_years and final_years

# Instantiate the regression model
input_size = 30
model = HybridModel(input_size)

# Define a loss function (Mean Squared Error) and an optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    output_temp = model(prev_years[0])
    output_distator = model(prev_years[1])
    
    output = tf.concat([output_temp.detach(), output_distator.detach()], axis=0)
    
    loss = criterion(output_temp, final_years[0])
    loss.backward()
    optimizer.step()

# After training, you can use the model to make predictions
model.eval()
for i in range(10):
    with torch.no_grad():
        test_input = prev_years[0]
        predicted_output = model(test_input)

    print(f"The predicted value is: {predicted_output.item()}")
    print(f"The actual value is: {final_years[i]}")
    print(abs(predicted_output.item()-final_years[i][0]))


The predicted value is: 2.0091625439895253
The actual value is: tensor([2.0120], dtype=torch.float64)
tensor(0.0028, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([1.5180], dtype=torch.float64)
tensor(0.4912, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([1.6880], dtype=torch.float64)
tensor(0.3212, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([1.2560], dtype=torch.float64)
tensor(0.7532, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([1.2120], dtype=torch.float64)
tensor(0.7972, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([0.8390], dtype=torch.float64)
tensor(1.1702, dtype=torch.float64)
The predicted value is: 2.0091625439895253
The actual value is: tensor([0.7700], dtype=torch.float64)
tensor(1.2392, dtype=torch.float64)
The predicted value is: 2.00916254

In [8]:
import numpy as np
import tensorflow as tf
from qiskit import QuantumCircuit, transpile, assemble, Aer
from sklearn.model_selection import train_test_split

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Define a quantum circuit
n_qubits = 4  # Number of qubits in the circuit (adjust as needed)
n_layers = 2  # Number of quantum layers (adjust as needed)

def create_quantum_circuit():
    circuit = QuantumCircuit(n_qubits)

    for layer in range(n_layers):
        for qubit in range(n_qubits):
            # Apply parameterized quantum gates (e.g., RY)
            theta = np.random.rand() * 2 * np.pi  # Random initial parameter value
            circuit.ry(theta, qubit)

        # Add entangling gates (e.g., CNOT)
        for i in range(n_qubits - 1):
            circuit.cx(i, i + 1)
    
    return circuit

# Create a quantum layer
quantum_circuit = create_quantum_circuit()

# Define the quantum feature map
def quantum_feature_map(x, quantum_circuit):
    quantum_circuits = []
    for x_value in x:
        circuit = quantum_circuit.copy()
        for qubit in range(n_qubits):
            theta = np.random.rand() * 2 * np.pi  # Random parameter values
            circuit.data[qubit][0].params[0] = theta
        quantum_circuits.append(circuit)
    
    state_vectors = []

    # Simulate the circuits and collect state vectors
    for circuit in quantum_circuits:
        simulator = Aer.get_backend('aer_simulator')
        job = execute(circuit, simulator)
        result = job.result()
        state_vector = result.get_statevector()
        state_vectors.append(np.abs(state_vector) ** 2)
    
    return state_vectors

# Convert the training and test data using the quantum feature map
x_train_quantum = np.array([quantum_feature_map(x, quantum_circuit) for x in x_train])
x_test_quantum = np.array([quantum_feature_map(x, quantum_circuit) for x in x_test])

# Build a classical neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2 ** n_qubits,)),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 classes for MNIST
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train_quantum, y_train, epochs=5, validation_data=(x_test_quantum, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test_quantum, y_test)
print(f"Test accuracy: {test_accuracy}")


QiskitError: 'No statevector for experiment "None"'